In [1]:
import json
data_file=open("D:\\ml_ve\\Dataset\\freego.json").read()
#print(data_file)

In [2]:
intents=json.loads(data_file)
print(intents)

{'intents': [{'tag': 'general', 'patterns': ['your current location'], 'responses': ['kannur', 'kochi', 'trivadrum', 'None'], 'answer': [1], 'context': ['']}, {'tag': 'general', 'patterns': ['what is the required age limit of an employe ?'], 'responses': ['18-20', '21-27', '28-33', '33 & above'], 'answer': [1], 'context': ['']}, {'tag': 'general', 'patterns': ['communication skill required by the employe ?'], 'responses': ['Excellent', 'Good', 'Average', 'Not much'], 'answer': [0], 'context': ['']}, {'tag': 'general', 'patterns': ['languages known by the employe ?'], 'responses': ['english', 'malayalam', 'Hindi', 'tamil'], 'answer': [0], 'context': ['']}, {'tag': 'general', 'patterns': ['how much percentage in the acadamic of the candidate are you looking for ?'], 'responses': ['90% &Above', '80% &Above', '70% &Above', '60% & Above'], 'answer': [1], 'context': ['']}, {'tag': 'general', 'patterns': ['which shift you are willing to work'], 'responses': ['morning', 'evening', 'night', 'fu

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
words=[]
classes=[]
documents=[]
questions=[]

ignore_words=["?","__________"]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print(pattern)
        questions.append(pattern)
        w=nltk.word_tokenize(pattern)
        #print(w)
        words.extend(w)
#print(words)
        documents.append((w,intent['tag']))
#print(documents)
        classes.append(intent['tag'])
#print(classes)
#print(len(words))
#print(documents)
#print(words)
#print(questions)

In [4]:
answers=[]
for intent in intents['intents']:
    for answer in intent['answer']:
        answers.append(answer)
print(answers)


[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 3, 3, 2, 0, 3, 2, 0, 3, 3, 0, 0, 3, 0, 1, 3, 3, 3, 2, 2, 0]


In [5]:
questions[8]

'how much would you be willing to pay the employee?'

In [6]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
#print(len(words))
#print(words)

In [7]:
words=sorted(list(set(words)))
#print(len(words))
print(words)

["'s", '(', ')', ',', '.', '2', '[', ']', 'a', 'about', 'acadamic', 'age', 'allowing', 'an', 'and', 'angular', 'angular2', 'any', 'app', 'application', 'are', 'attribute', 'automatically', 'available', 'backend', 'be', 'benefit', 'binding', 'built-in', 'by', 'can', 'candidate', 'change', 'child', 'code', 'coding', 'communication', 'company', 'component', 'conditional', 'correct', 'current', 'data', 'defined', 'defines', 'described', 'detect', 'developer', 'device', 'directive', 'disclose', 'displayed', 'do', 'duplication', 'easier', 'easily', 'element', 'employe', 'employee', 'entire', 'event', 'expected', 'experience', 'expression', 'feature', 'following', 'for', 'from', 'frontend', 'how', 'href', 'in', 'integrates', 'intented', 'is', 'it', 'job', 'known', 'language', 'lie', 'limit', 'link', 'location', 'looking', 'make', 'many', 'marked', 'match', 'mention', 'meta-programming', 'method', 'mobile', 'much', 'native', 'nativescript', 'needed', 'new', 'ngswitchwhen', 'no', 'normal', 'of'

In [8]:
classes=sorted(list(set(classes)))
#print(len(classes))
#print(classes)

In [9]:
print("no: of words:",len(words))
print("no: of documents:",len(documents))
print("no: of classes:",len(classes))

no: of words: 178
no: of documents: 30
no: of classes: 2


In [10]:
import pickle
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
pickle.dump(questions,open('questions.pkl','wb'))
pickle.dump(answers,open('answers.pkl','wb'))

In [11]:
training=[]
output_empty=[0]*len(classes)
#print(output_empty)

In [12]:
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    #print(pattern_words)
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #print(pattern_words)
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #print(len(bag))
    #print(bag)
    output_row=list(output_empty)
    #print(output_row)
    output_row[classes.index(doc[1])]=1
    #print(output_row)
    training.append([bag,output_row])
    #print(training)

In [13]:
import numpy as np
import random
random.shuffle(training)
training=np.array(training)
#print(training)

<ipython-input-13-3cd37e702f2b>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training=np.array(training)


In [14]:
train_x=list(training[:,0])
train_y=list(training[:,1])
print(len(train_y[0]))
print(len(train_x[0]))

2
178


In [15]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import SGD

model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)
print("model created")

Epoch 1/200
6/6 [==============================] - 17s 1ms/step - loss: 0.6606 - accuracy: 0.6410
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 0.5829 - accuracy: 0.7681
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 0.6246 - accuracy: 0.6567
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 0.4668 - accuracy: 0.8610
Epoch 5/200
6/6 [==============================] - 0s 1ms/step - loss: 0.4819 - accuracy: 0.7995
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 0.3088 - accuracy: 0.9457
Epoch 7/200
6/6 [==============================] - 0s 1ms/step - loss: 0.3069 - accuracy: 0.8995
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 0.3532 - accuracy: 0.8724
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1355 - accuracy: 0.9752
Epoch 10/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1121 - accuracy: 1.0000
Epoch 11/200
6/6 [==========

6/6 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 87/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 997us/step - loss: 8.6154e-04 - accuracy: 1.0000
Epoch 89/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 91/200
6/6 [==============================] - 0s 991us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 92/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 93/200
6/6 [==============================] - 0s 989us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 94/200
6/6 [=====

6/6 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 166/200
6/6 [==============================] - 0s 1ms/step - loss: 7.3930e-05 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 997us/step - loss: 5.5313e-04 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 997us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 1ms/step - loss: 3.4435e-04 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 1ms/step - loss: 7.0600e-05 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 996us/step - loss: 6.8191e-04 - accuracy: 1.000

In [16]:
x_test=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
model.predict([x_test])[0]

array([9.990940e-01, 9.060572e-04], dtype=float32)

In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open("D:\\ml_ve\\Dataset\\freego.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
questions=pickle.load(open('questions.pkl','rb'))
answers=pickle.load(open('answers.pkl','rb'))
print(answers)

[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 3, 3, 2, 0, 3, 2, 0, 3, 3, 0, 0, 3, 0, 1, 3, 3, 3, 2, 2, 0]


In [18]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [19]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [20]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.75
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [21]:
def getResponse(ints, intents_json,text):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    t=[]
    t.append(text)
    for i in list_of_intents:
        if(i['tag']== tag) and (i['patterns']==t):
            result = i['responses']
            break
    return result

In [22]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents,text)
    return res

In [34]:
import tkinter
from tkinter import *
from PIL import ImageTk,Image

indexes=[]
user_answers=[]
def indexgen():
    global indexes
    while(len(indexes)<21):
        x=random.randint(0,29)
        if x in indexes:
            continue
        else:
            indexes.append(x)

def backend(score):
    labelquestion.destroy()
    r1.destroy()
    r2.destroy()
    r3.destroy()
    r4.destroy()
    
    labelheading=Label(root,text="Your Result",font=("comic sans MS",18,"bold"),background="#ffffff")
    labelheading.pack()
    
    img3=ImageTk.PhotoImage(Image.open("passed.png"))
    img4=ImageTk.PhotoImage(Image.open("failed.png"))
    
    labelresultimg=Label(root)
    labelresultimg.pack(pady=(100,0))
    
    labelresulttext=Label(root,font=("Consolas",18,"bold"))
    labelresulttext.pack(pady=(0,10))
    
    if(score>=75):
        labelresultimg.configure(image=img3,background="#ffffff")
        labelresultimg.image=img3
        labelresulttext.configure(text="Congragulation!!!\nYou have Passed",background="#ffffff")
    else:
        labelresultimg.configure(image=img4,background="#ffffff")
        labelresultimg.image=img4
        labelresulttext.configure(text="Sorry!!!\nYou have Failed\nBetter Luck next Time",background="#ffffff")
        
    labelscore=Label(root,text=("Total Score:",score),font=("Consolas",14))
    labelscore.pack()



def calScore():
    global indexes,user_answers,answers
    x=0
    score=0
    for i in indexes:
        if user_answers[x]==answers[i]:
            score = score+5
        x+=1
    backend(score)
        
        

ques=1           
def selected():
    global radiovar,user_answers
    global labelquestion,r1,r2,r3,r4
    global ques
    x=radiovar.get()
    user_answers.append(x)
    radiovar.set(-1)
    if(ques<21):
        msg=questions[indexes[ques]]
        msg1="Q:) " + msg
        labelquestion.config(text=msg1)
        res=chatbot_response(msg)
        r1["text"]=res[0] 
        r2["text"]=res[1]
        r3["text"]=res[2]
        r4["text"]=res[3]
        ques+=1
    else:
        calScore()

def startquiz():
    global labelquestion,r1,r2,r3,r4
    msg=questions[indexes[0]]
    msg1="Q:) " + msg
    labelquestion=Label(root,text=msg1,font=("Consolas",14),width=500,justify="center",wraplength=400)
    labelquestion.pack(pady=(100,50))
    global radiovar
    radiovar = IntVar()
    radiovar.set(-1)
    res=chatbot_response(msg)
    r1=Radiobutton(root,text=res[0],font=("Times",12),value=0,variable=radiovar,justify="center",background="#ffffff",command=selected)
    r1.pack(pady=(0,10))
    
    r2=Radiobutton(root,text=res[1],font=("Times",12),value=1,variable=radiovar,justify="center",background="#ffffff",command=selected)
    r2.pack(pady=(0,10))
    
    r3=Radiobutton(root,text=res[2],font=("Times",12),value=2,variable=radiovar,justify="center",background="#ffffff",command=selected)
    r3.pack(pady=(0,10))
    
    r4=Radiobutton(root,text=res[3],font=("Times",12),value=3,variable=radiovar,justify="center",background="#ffffff",command=selected)
    r4.pack(pady=(0,10))
        

def start():
    labelimage.destroy()
    labeltext.destroy()
    labelinstruction.destroy()
    labelrules.destroy()
    btnstart.destroy()
    indexgen()
    startquiz()
    
    
    
root=tkinter.Tk()
root.title("FreegoQuiz")
root.geometry("700x600")
root.config(background="#ffffff")
#root.resizable(0,0)

img1=ImageTk.PhotoImage(Image.open("freego.png"))
labelimage=Label(root,image=img1,background="#ffffff")
labelimage.pack(pady=(50,0))

labeltext=Label(root,text="Interview",font=("comic sans MS",18,"bold"),background="#ffffff")
labeltext.pack(pady=(0,60))

img2=ImageTk.PhotoImage(Image.open("start.png"))

btnstart=Button(root,image=img2,relief=FLAT,border=0,background="#ffffff",command=start)
btnstart.pack()

labelinstruction=Label(root,text="Read the Rules below And\nClick Start once You are Ready",background="#ffffff",font=("Consolas",10),justify="center")
labelinstruction.pack(pady=(0,150))

labelrules=Label(root,text="1. Read the questions clearly and answer\n2. Once the option is selected it trigger next question so select the option wisely\n3. You only have one chance for each questions\n4. There is no negative score for wrong answer",width=150,background="#000000",foreground="#ffffff",font=("Times",13))
labelrules.pack()

root.mainloop()